In [1]:
from aiy.vision.inference import CameraInference
from aiy.vision.models import object_detection
from aiy.vision.streaming.server import StreamingServer
from aiy.vision.streaming import svg
from aiy.leds import Leds, Color
from gpiozero import Servo
from aiy.pins import PIN_A

from picamera import PiCamera
from IPython.display import Image, display, clear_output

import aiy.toneplayer
import contextlib
import time

### Joy Meter Demo

We are going to use the servo motor to indicate the level of joy detected. Feel free to tape a cardboard arrow to the servo arm or create a dail :) And remember if your joy detector is running, you need to turn it off using the commands

```
sudo systemctl stop joy_detection_demo.service
```

The code below initializes the servo assuming you connected it to PIN_A. Ignore the notics when runnig the code. Also it **seems the only way to run this cell twice is to restart the kernel** (still debugging), so run it once and then change and play wioth the code below.

In [2]:
servo = Servo(PIN_A)

/usr/lib/python3/dist-packages/gpiozero/output_devices.py:1533: PWMSoftwareFallback: To reduce servo jitter, use the pigpio pin factory.See https://gpiozero.readthedocs.io/en/stable/api_output.html#servo for more info
  'To reduce servo jitter, use the pigpio pin factory.'


I added some code here that is called to create the overlay - basically the box around the face and the score above. You can customize it and/or add information you want to overlay on the camera feed. **This is not needed for the servo.**

In [3]:
def svg_overlay(objects, frame_size):


    width, height = frame_size
    doc = svg.Svg(width=width, height=height)

    for obj in objects:
        x, y, w, h = obj.bounding_box
        fcol='rgb'+str(Color.RED)
        doc.add(svg.Rect(x=int(x), y=int(y), width=int(w), height=int(h), rx=10, ry=10,
                         fill_opacity=0.3,
                         style='fill:'+fcol+';stroke:white;stroke-width:4px'))
        
        if obj.kind == 1:
            doc.add(svg.Text("⚠️INTRUDER⚠️", x=x, y=y-10, fill='red', font_size=30))
        elif obj.kind == 2:
            doc.add(svg.Text("frend 🐱", x=x, y=y-10, fill='yellow', font_size=30))
        elif obj.kind == 3:
            doc.add(svg.Text("frend 🐶", x=x, y=y-10, fill='yellow', font_size=30))

    return str(doc)

#### Main loop

Here is our main loop based on the code we used last time. Look at the comments on what was changed. Basically we removed the part saving the picture and adjust the servo value to the joy score each frame. We also added the streaming back in, to while this cell runs, you can connect to http://orcspi-vis.local:4664 and see the stream.

In [5]:
with contextlib.ExitStack() as stack:
    leds   = stack.enter_context(Leds())
    camera = stack.enter_context(PiCamera(sensor_mode=4, resolution=(820, 616)))
    player = aiy.toneplayer.TonePlayer(22)
    
    model_loading = ['Cq', 'Dq', 'Eq']
    person = ['Cq']

    # This starts and runs the streaming of the camera
    server = stack.enter_context(StreamingServer(camera))  
    
    player.play(*model_loading)
    print ("Loading model - hold on ..")
    
    # Move the servo to low (-1)
    servo.min()
    
    # Do inference on VisionBonnet
    with CameraInference(object_detection.model()) as inference:
        try:   
            for result in inference.run():
                objects = object_detection.get_objects(result)
                
                # This sends the overlay (boxes) to add to the camera stream
                server.send_overlay(svg_overlay(objects, (result.width, result.height)))

                if len(objects) >= 1:
                    obj = ["person", "cat", "dog"]
                    clear_output(wait=True) 
                    print("Objects:")
                    for o in objects:
                        print("- ", obj[o.kind - 1]) 
                        
                    if obj[o.kind - 1] == "person":
                        player.play(*person)
                        leds.update(Leds.rgb_on(Color.RED))
                    elif obj[o.kind - 1] == "cat":
                        leds.update(Leds.rgb_on(Color.BLUE))
                    elif obj[o.kind - 1] == "dog":
                        leds.update(Leds.rgb_on(Color.GREEN))  
                    else:
                        leds.update(Leds.rgb_on(Color.BLACK))   
                        
                        
                    # leds.update(Leds.rgb_on(Color.blend(Color.YELLOW, Color.RED, faces[0].joy_score)))
                    
                    # This sets the servo to the joy score value
                    # To use the full range (-1 to 1) we multiply the score by 2 and subtract 1
                    # servo.value = 2*faces[0].joy_score  - 1
                                           
        except KeyboardInterrupt:
            leds.update(Leds.rgb_off())
    
    # Servo back to the middle upon ending
    servo.mid()
    
    print("Done")

Objects:
-  person


[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Running behind, dropping messages
[10.0.0.70:52441] Error while processing websocket request
Traceback (most recent call last):
  File "/home/pi/AIY-projects-python/src/aiy/vision/streaming/server.py", line 570, in _receive_message
    packet = self._receive_packet()
  File "/home/pi/AIY-projects-python/src/aiy/vision/streaming/server.py", line 612, in _receive_packet
    packet.fin = buf[0] & 0x80 > 0
IndexError: index out of range


OSError: [Errno 62] Timer expired